# Topology - Persistent Homology

In [234]:
import MetaTrader5 as mt5
from datetime import datetime
import pytz
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import warnings
from sklearn.metrics.pairwise import euclidean_distances
import gudhi as gd
import shutil
import ta
warnings.filterwarnings("ignore")
mt5.initialize()
account=51127988
password= "Aar2frM7"
server = 'ICMarkets-Demo'

In [265]:
def get_rates(pair1, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, mt5.TIMEFRAME_M15, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1

AUDUSD = get_rates('AUDUSD.a', 250)

In [266]:
# Other data ideas:
# AUD basket, AUD cointegrating pairs, AUD correlated pairs
# arima values, garch values, arma values

In [267]:
df = AUDUSD[['open', 'high', 'low', 'close']]
df = df.iloc[:-1] 

In [268]:
df['tr1'] = df['high'] - df['low']
df['tr2'] = abs(df['high'] - df['close'].shift())
df['tr3'] = abs(df['low'] - df['close'].shift())
df['tr'] = df[['tr1', 'tr2', 'tr3']].max(axis=1)

In [269]:
from ta.trend import MACD
from ta.momentum import RSIIndicator, StochRSIIndicator
from ta.volatility import DonchianChannel

In [270]:
macd_indicator = MACD(df['close'])
df['MACD'] = macd_indicator.macd()
df['MACD_signal'] = macd_indicator.macd_signal()
df['MACD_diff'] = macd_indicator.macd_diff()

# RSI
rsi_indicator = RSIIndicator(df['close'])
df['RSI'] = rsi_indicator.rsi()

# StochRSI
stoch_rsi_indicator = StochRSIIndicator(df['close'])
df['StochRSI'] = stoch_rsi_indicator.stochrsi()
df['StochRSI_K'] = stoch_rsi_indicator.stochrsi_k()
df['StochRSI_D'] = stoch_rsi_indicator.stochrsi_d()

# Donchian Channel
df['middle'] = df[['high', 'low']].median(axis=1)
donchian_channel = DonchianChannel(df['high'], df['low'], df['close'])
df['Donchian_channel_high'] = donchian_channel.donchian_channel_hband()
df['Donchian_channel_low'] = donchian_channel.donchian_channel_lband()
df['Donchian_channel_middle'] = donchian_channel.donchian_channel_mband()

In [271]:
df['ma7'] = df['close'].rolling(window = 7).mean()
df['ma21'] = df['close'].rolling(window = 21).mean()
window_size = 7  # Choose the desired window size for the rolling calculation
df['atr'] = df['tr'].rolling(window=window_size).mean()
df = df.dropna()
df

,open,high,low,close,tr1,tr2,tr3,tr,MACD,MACD_signal,...,StochRSI,StochRSI_K,StochRSI_D,middle,Donchian_channel_high,Donchian_channel_low,Donchian_channel_middle,ma7,ma21,atr
33,0.65390,0.65394,0.65383,0.65384,0.00011,0.00004,0.00007,0.00011,-0.000043,-0.000046,...,0.000000,0.379310,0.376881,0.653885,0.65423,0.65357,0.653900,0.653889,0.653924,0.000069
34,0.65388,0.65391,0.65382,0.65386,0.00009,0.00007,0.00002,0.00009,-0.000043,-0.000046,...,0.273461,0.298676,0.379588,0.653865,0.65423,0.65372,0.653975,0.653881,0.653935,0.000076
35,0.65390,0.65400,0.65385,0.65400,0.00015,0.00014,0.00001,0.00015,-0.000032,-0.000043,...,1.000000,0.424487,0.367491,0.653925,0.65418,0.65372,0.653950,0.653899,0.653937,0.000086
36,0.65401,0.65405,0.65389,0.65390,0.00016,0.00005,0.00011,0.00016,-0.000030,-0.000040,...,0.410931,0.561464,0.428209,0.653970,0.65416,0.65372,0.653940,0.653903,0.653924,0.000100
37,0.65392,0.65397,0.65384,0.65386,0.00013,0.00007,0.00006,0.00013,-0.000032,-0.000039,...,0.188528,0.533153,0.506368,0.653905,0.65416,0.65380,0.653980,0.653899,0.653918,0.000114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,0.65062,0.65068,0.65022,0.65022,0.00046,0.00004,0.00042,0.00046,0.000463,0.000552,...,0.000000,0.229913,0.522582,0.650450,0.65098,0.64928,0.650130,0.650677,0.650369,0.000331
245,0.65021,0.65021,0.64964,0.64972,0.00057,0.00001,0.00058,0.00058,0.000369,0.000515,...,0.000000,0.121993,0.305051,0.649925,0.65098,0.64928,0.650130,0.650536,0.650364,0.000380
246,0.64972,0.64980,0.64926,0.64926,0.00054,0.00008,0.00046,0.00054,0.000254,0.000463,...,0.000000,0.000000,0.117302,0.649530,0.65098,0.64926,0.650120,0.650316,0.650311,0.000424
247,0.64926,0.64955,0.64890,0.64912,0.00065,0.00029,0.00036,0.00065,0.000150,0.000401,...,0.000000,0.000000,0.040664,0.649225,0.65098,0.64890,0.649940,0.650069,0.650253,0.000480


In [272]:
# Compute the Euclidean distance matrix
dist_matrix = euclidean_distances(df)

# Use the GUDHI library to construct a Rips complex
rips_complex = gd.RipsComplex(distance_matrix=dist_matrix, max_edge_length=1.0)

simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)

# To visualize or analyze the topological features of the complex, you can use GUDHI's persistence diagram functionality:
diag = simplex_tree.persistence(min_persistence=0.01)

In [285]:
# Set a threshold for the minimum persistence that we consider significant
min_persistence = 0.1
threshold_high = 20
threshold_low = 5

# Calculate the persistence diagram
diag = simplex_tree.persistence(min_persistence=min_persistence)

# Extract the persistent features (those with persistence above the threshold)
persistent_features = [interval for interval in diag if interval[1][1] - interval[1][0] > min_persistence]

# Hypothetical trading logic
if len(persistent_features) > threshold_high:
    print("The market is complex, potentially indicating a trend change. Short")
    if len(mt5.positions_get()) == 0:
        sell_order()
    else:
        for i in mt5.positions_get():
            if 'Rips' in i.comment and i.type == 1:
                print('Already in short position. No action')
                # position = i 
                # close_position(position) ## still closing everything else, amend when important
                # sell_order()
            elif 'Rips' in i.comment and i.type == 0:
                print('Previously long. Changing to short.')
                position = i 
                close_position(position) ## still closing everything else, amend when important
                sell_order()

elif len(persistent_features) < threshold_low:
    print("The market is simple, potentially indicating a trend continuation. Buy")
    if len(mt5.positions_get()) == 0:
        buy_order()
    else:
        for i in mt5.positions_get():
            if i.type == 1:
                print('Already long')
                break
            else:
                close_position(i)
                buy_order()
else:
    print("The market is in an intermediate state. No action taken.")

The market is complex, potentially indicating a trend change. Short
Already in short position. No action


In [71]:
def buy_order():
    price = mt5.symbol_info_tick('AUDUSD.a').ask
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": 'AUDUSD.a',
        "volume": 1.00,
        "type": mt5.ORDER_TYPE_BUY,
        "price": price,
        "deviation": 20,
        "magic": 234000,
        "comment": f"Rips Complex",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result1 = mt5.order_send(request)
    result1
    
def sell_order():
    price = mt5.symbol_info_tick('AUDUSD.a').bid
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": 'AUDUSD.a',
        "volume": 1.00,
        "type": mt5.ORDER_TYPE_SELL,
        "price": price,
        "deviation": 20,
        "magic": 234000,
        "comment": f"Rips Complex",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result1 = mt5.order_send(request)
    result1
    
def close_position(position):

    tick = mt5.symbol_info_tick(position.symbol)

    request = {
        "action" : mt5.TRADE_ACTION_DEAL,
        "position": position.ticket,
        "symbol": position.symbol,
        "volume": position.volume,
        "type": mt5.ORDER_TYPE_BUY if position.type == 1 else mt5.ORDER_TYPE_SELL,
        "price": tick.ask if position.type == 1 else tick.bid,
        "deviation": 20,
        "magic": 100,
        "comment": 'pytohn script close',
        'type_time': mt5.ORDER_TIME_GTC,
        'type_filling':mt5.ORDER_FILLING_IOC,

        }
    result1 = mt5.order_send(request)